In [ ]:
import psycopg2
try:
    # get secret values from keyvault
    postgres_pw = TokenLibrary.getSecret("redacted", "redacted", "redacted")
    # make connection
    conn = psycopg2.connect(dbname="rdt_dev",user="redacted",host="redacted",password=postgres_pw)
    conn.autocommit = True
    cur=conn.cursor()
    print("Database connected")
except:
    print("I am unable to connect to the database")

In [ ]:
sql=f'''CREATE SCHEMA IF NOT EXISTS metadata;
CREATE TABLE IF NOT EXISTS metadata.tableupdates (
    dbname      VARCHAR(255),
    schemaname  VARCHAR(255),
    tablename   VARCHAR(255),
    last_update TIMESTAMPTZ,
    scriptname  VARCHAR(255),
    updatetype  VARCHAR(255),
    primary key(dbname, schemaname, tablename)
);'''
cur.execute(sql)

In [ ]:
sql=f'''CREATE PROCEDURE update_metadata(var_dbname VARCHAR, var_schemaname VARCHAR, var_tablename VARCHAR, var_last_update TIMESTAMPTZ, var_scriptname VARCHAR, var_updatetype VARCHAR)
LANGUAGE SQL
AS $$
    INSERT INTO metadata.tableupdates (dbname, schemaname, tablename, last_update, scriptname, updatetype)
    VALUES      (var_dbname, var_schemaname, var_tablename, var_last_update, var_scriptname, var_updatetype)
    ON CONFLICT (dbname, schemaname, tablename)
    DO UPDATE SET
        last_update = EXCLUDED.last_update
        ,scriptname = EXCLUDED.scriptname
        ,updatetype = EXCLUDED.updatetype;
$$;'''
cur.execute(sql)

In [ ]:
# Example update call by scripts
# sql_meta='''CALL update_metadata('rdt_dev','stg_...','...',NOW(),'notebookname','calculation');'''
# cur.execute(sql_meta)